# Method to process atoms object into a more finalized form
___

The following operations will be performed:
1. Atoms will be wrapped to be within the unit cell
2. Atoms will be centered w.r.t. the z-axis
3. Appropriate amount of vacuum will be added
4. Lower half of support atoms will be constrained

# Import Modules

In [1]:
import copy

import numpy as np

from ase import io
from ase.visualize import view
from ase.constraints import FixAtoms

# Script Inputs

In [2]:
max_thickness = 4

z_vacuum = 13

frac_constr_supp = 0.6

# Read Single Atoms Object

In [3]:
atoms_i = io.read("temp.traj")

view(atoms_i)

# Wrap atoms into unit cell

In [4]:
atoms_i.wrap()

# Mirror atoms about xy-plane if Graphene layer is on the bottom

In [5]:
c_z_pos = np.mean(np.array([i.position[-1] for i in atoms_i if i.symbol == "C"]))
m_z_pos = np.mean(np.array([i.position[-1] for i in atoms_i if i.symbol != "C"]))

if c_z_pos < m_z_pos:
    print("Graphene is below support")

    new_positions = []
    for pos_i in atoms_i.positions:
        pos_i[-1] = -1 * pos_i[-1]
        new_positions.append(pos_i)
    atoms_i.set_positions(new_positions)
    atoms_i.center()

# Add z-space vacuum

In [6]:
z_new = 50.
new_cell = copy.copy(atoms_i.cell)
new_cell[-1][-1] = z_new
atoms_i.set_cell(new_cell)

atoms_i.center()

# Set slab thickness

In [7]:
max_slab_z = np.array([i.position[-1] for i in atoms_i if i.symbol != "C"]).max()
min_slab_z = np.array([i.position[-1] for i in atoms_i if i.symbol != "C"]).min()
# print(max_slab_z)
# print(min_slab_z)

atoms_to_delete_index_list = []
for atom_j in atoms_i:
    if atom_j.position[-1] < max_slab_z - max_thickness:
        atoms_to_delete_index_list.append(atom_j.index)
del atoms_i[atoms_to_delete_index_list]

# Constrain lower part of support

In [8]:
max_slab_z = np.array([i.position[-1] for i in atoms_i if i.symbol != "C"]).max()
min_slab_z = np.array([i.position[-1] for i in atoms_i if i.symbol != "C"]).min()

c_atoms_i = atoms_i[[i.index for i in atoms_i if i.symbol == "C"]]
m_atoms_i = atoms_i[[i.index for i in atoms_i if i.symbol != "C"]]

frac_constr_supp = 0.55
z_cutoff = (max_slab_z - min_slab_z) * frac_constr_supp + min_slab_z

constraints_index_list = [i.index for i in m_atoms_i if i.position[-1] < z_cutoff]

c = FixAtoms(indices=[atom.index for atom in atoms_i if atom.index in constraints_index_list])
atoms_i.set_constraint(c)

# Set z-spacing again

In [10]:
new_cell = copy.copy(atoms_i.cell)

# Added 2 to account for still unknown graphene spacing
new_cell[-1][-1] = z_vacuum + (max_slab_z - min_slab_z + 2)
atoms_i.set_cell(new_cell)

atoms_i.center()

In [11]:
view(atoms_i)

In [12]:
assert False

AssertionError: 

In [ ]:
# atoms_i.center()
# atoms_i.wrap()
# atoms_i.center()


# #     atoms_i = copy.deepcopy(atoms_i)

# z_pos_list = []
# for atom_j in atoms_i:   
#     if atom_j.symbol != "C":
#         z_pos_list.append(atom_j.position[2])
# z_pos_list = np.array(z_pos_list)


# max_slab_z = z_pos_list.max()

# atoms_to_delete_index_list = []
# for atom_j in atoms_i:
#     if atom_j.position[-1] < max_slab_z - max_thickness:
#         atoms_to_delete_index_list.append(atom_j.index)


In [ ]:
# new_positions = []
# for pos_i in atoms_i.positions:
#     pos_i[-1] = -1 * pos_i[-1]

#     new_positions.append(pos_i)

# atoms_i.set_positions(new_positions)

# atoms_i.center()

In [ ]:
# new_positions = []
# for pos_i in atoms_i.positions:
#     pos_i[-1] = -1 * pos_i[-1]
#     new_positions.append(pos_i)
# atoms_i.set_positions(new_positions)

# atoms_i.center()